<a href="https://colab.research.google.com/github/jzhangfob/igdb-games-data-pipeline/blob/feature%2Fstaging-data-pipeline/notebooks/Twitch-Data-Staging-Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialization

In [10]:
# Import packages
import requests
import csv
import time
import pandas as pd
import numpy as np
import os
import json

from google.cloud import storage
from io import StringIO

In [11]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Set the google application credentials path after uploading the service account key to Google Drive
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/Twitch Data Pipeline/igdb-pipeline-a3bbac471b4c.json"

In [13]:
# Define the bucket name to loop through
BUCKET_NAME = "igdb_raw_data_bucket"

## Functions

In [14]:
# Create a list of blobs (Files from buckets)
def raw_data_ingest(bucket_name):
  """
  Add GCS bucket files to a dict and return it
  Parameters:
  -----------
  bucket_name (string): Global variable of the bucket name to retrieve source data

  Returns:
  -----------
  raw_df: dictionary storing raw data
  """
  # Initialize the GCS client
  client = storage.Client()
  # List of blobs/files from bucket
  blobs = client.list_blobs(bucket_name)
  # Returns an empty dict with raw data
  raw_df = {}

  # Loop through each blob object
  for blob in blobs:

    # Get the name and data from each blob
    blob_name = blob.name
    data = blob.download_as_text()

    # Add blobs as key/value entries into raw_df
    if blob_name not in raw_df.keys():
      raw_df[blob_name] = pd.read_csv(StringIO(data))

  return raw_df

In [15]:
raw_data = raw_data_ingest(BUCKET_NAME)

<ipython-input-14-f32e5355b5ca>:29: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df[blob_name] = pd.read_csv(StringIO(data))


## Cleansing

```
# Cleanse raw data to create dim and fact tables
```



In [95]:
# Ingest raw data
games_df = raw_data['games']
external_games_df = raw_data['external_games']
game_engines_df = raw_data['game_engines']
genres_df = raw_data['genres']
platforms_df = raw_data['platforms']
game_modes_df = raw_data['game_modes']

In [56]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296516 entries, 0 to 296515
Data columns (total 56 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       296516 non-null  int64  
 1   age_ratings              65865 non-null   object 
 2   alternative_names        66500 non-null   object 
 3   category                 296516 non-null  int64  
 4   cover                    231491 non-null  float64
 5   created_at               296516 non-null  int64  
 6   external_games           274020 non-null  object 
 7   first_release_date       202896 non-null  float64
 8   game_modes               174461 non-null  object 
 9   genres                   244385 non-null  object 
 10  involved_companies       135168 non-null  object 
 11  keywords                 100929 non-null  object 
 12  name                     296515 non-null  object 
 13  platforms                216936 non-null  object 
 14  play

In [96]:
# Filter columns
games_df = games_df[['id',
                     'name',
                     'first_release_date',
                     'created_at',
                     'aggregated_rating',
                     'aggregated_rating_count',
                     'rating',
                     'rating_count',
                     'game_engines',
                     'game_modes',
                     'genres',
                     'platforms',
                     'updated_at',
                     'external_games',
                     'category']].rename(columns={
                         'id':'game_id',
                         'name':'game_name',
                         'aggregated_rating':'critic_rating',
                         'aggregated_rating_count':'critic_rating_count',
                         'rating':'igdb_rating',
                         'rating_count':'igdb_rating_count',
                         'game_engines':'game_engine_id',
                         'game_modes':'game_mode_id',
                         'genres':'genre_id',
                         'platforms':'platform_id',
                         'external_games':'external_game_id'
                     })

# Filter for only main games
games_df = games_df[games_df['category'] == 0].reset_index(drop=True)

In [97]:
external_games_df = external_games_df[['id',
                                       'game',
                                       'uid',
                                       'category',
                                       'created_at']].rename(columns={
                                           'id':'external_game_id',
                                           'game':'game_id',
                                           'uid':'steam_app_id',
                                       })

external_games_df = external_games_df[external_games_df['category'] == 1].reset_index(drop=True)

In [98]:
external_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126975 entries, 0 to 126974
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   external_game_id  126975 non-null  int64 
 1   game_id           126975 non-null  int64 
 2   steam_app_id      126975 non-null  object
 3   category          126975 non-null  int64 
 4   created_at        126975 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 4.8+ MB


In [99]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250932 entries, 0 to 250931
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   game_id              250932 non-null  int64  
 1   game_name            250931 non-null  object 
 2   first_release_date   167754 non-null  float64
 3   created_at           250932 non-null  int64  
 4   critic_rating        13196 non-null   float64
 5   critic_rating_count  13196 non-null   float64
 6   igdb_rating          27811 non-null   float64
 7   igdb_rating_count    27811 non-null   float64
 8   game_engine_id       26987 non-null   object 
 9   game_mode_id         146272 non-null  object 
 10  genre_id             204844 non-null  object 
 11  platform_id          180233 non-null  object 
 12  updated_at           250932 non-null  int64  
 13  external_game_id     246340 non-null  object 
 14  category             250932 non-null  int64  
dtypes: float64(5), in

In [105]:
external_games_df[external_games_df['game_id'] == 133405]

,external_game_id,game_id,steam_app_id,category,created_at
35116,1863447,133405,1230170,1,1588631163
83514,2522133,133405,1914150,1,1658766929
107260,1725019,133405,1101120,1,1560479321


In [104]:
merged[merged['game_id']==133405]

,game_id,game_name,first_release_date,created_at_x,critic_rating,critic_rating_count,igdb_rating,igdb_rating_count,game_engine_id,game_mode_id,genre_id,platform_id,updated_at,external_game_id_x,category_x,external_game_id_y,steam_app_id,category_y,created_at_y
61316,133405,Gunsmith Simulator,1.728000e+09,1589042462,NaN,NaN,NaN,NaN,NaN,[1],"[5, 13, 15, 24]",[6],1731575465,"[1725019, 1863447, 1986063, 2522133]",0,1863447.0,1230170,1.0,1.588631e+09
61317,133405,Gunsmith Simulator,1.728000e+09,1589042462,NaN,NaN,NaN,NaN,NaN,[1],"[5, 13, 15, 24]",[6],1731575465,"[1725019, 1863447, 1986063, 2522133]",0,2522133.0,1914150,1.0,1.658767e+09
61318,133405,Gunsmith Simulator,1.728000e+09,1589042462,NaN,NaN,NaN,NaN,NaN,[1],"[5, 13, 15, 24]",[6],1731575465,"[1725019, 1863447, 1986063, 2522133]",0,1725019.0,1101120,1.0,1.560479e+09


In [102]:
external_games_df['game_id'].value_counts()

,count
game_id,
133405,3
114840,3
139609,3
151060,3
139192,3
...,...
321949,1
124398,1
295782,1


In [100]:
merged = pd.merge(games_df, external_games_df, how='left', on='game_id')

In [101]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252802 entries, 0 to 252801
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   game_id              252802 non-null  int64  
 1   game_name            252801 non-null  object 
 2   first_release_date   169143 non-null  float64
 3   created_at_x         252802 non-null  int64  
 4   critic_rating        13378 non-null   float64
 5   critic_rating_count  13378 non-null   float64
 6   igdb_rating          28062 non-null   float64
 7   igdb_rating_count    28062 non-null   float64
 8   game_engine_id       27509 non-null   object 
 9   game_mode_id         147691 non-null  object 
 10  genre_id             206678 non-null  object 
 11  platform_id          181750 non-null  object 
 12  updated_at           252802 non-null  int64  
 13  external_game_id_x   248210 non-null  object 
 14  category_x           252802 non-null  int64  
 15  external_game_id_

## Exploration

Dev environment

In [17]:
external_games_df[external_games_df['uid'] == '892970']

,id,category,created_at,game,name,uid,updated_at,url,year,checksum
77464,1602828,1,1538813824,104967,Valheim,892970,1631804512,https://store.steampowered.com/app/892970,0.0,83903784-3e21-0f72-0cf8-07eb852461e8


In [34]:
not_in_games = external_games_df[~external_games_df['id'].isin(games_df['id'])]

In [47]:
not_in_external_games = games_df[~games_df['id'].isin(external_games_df['id'])]

In [49]:
external_games_df[external_games_df.id == 91579]

,id,category,created_at,game,name,uid,updated_at,url,year,checksum


In [61]:
games_df.head()

,game_id,game_name,first_release_date,created_at,critic_rating,critic_rating_count,igdb_rating,igdb_rating_count,game_engine_id,game_mode_id,genre_id,platform_id,updated_at,external_game_id,category
0,131913,Maji Kyun! Renaissance,1.474416e+09,1584788069,NaN,NaN,NaN,NaN,NaN,[1],[34],[46],1681992241,[1977891],0
1,5668,Commando,4.812480e+08,1396151250,80.0,1.0,71.753318,47.0,NaN,"[1, 2]",[5],"[5, 13, 15, 16, 25, 26, 27, 52, 60, 63, 67, 69...",1730080562,"[16380, 150790, 1187191]",0
2,88308,Hey Duggee: The Big Outdoor App,1.478650e+09,1519237439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[39],1685014987,"[243459, 1960113]",0
3,165668,Endings,NaN,1630482010,NaN,NaN,NaN,NaN,NaN,NaN,[31],NaN,1685764902,"[2084097, 2115290]",0
4,95080,Dotra,NaN,1521818623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1670992528,[1989881],0


In [83]:
merged = pd.merge(games_df, external_games_df, how='left', on='game_id')

In [90]:
external_games_df[(external_games_df['game']==154986) & (external_games_df['category']== 1)]

,id,category,created_at,game,name,uid,updated_at,url,year,checksum
298757,2072634,1,1626102117,154986,FIFA 22,1506830,1626107540,https://store.steampowered.com/app/1506830,2021.0,ff3226e5-e484-4622-7763-f52efbf87fc7


In [87]:
external_games_df['game'].value_counts()

,count
game,
154986,93
205780,78
242493,76
256092,68
207393,67
...,...
168487,1
168484,1
168482,1


In [85]:
games_df['game_id'].value_counts()

,count
game_id,
89324,2
131913,1
48771,1
171155,1
195145,1
...,...
18450,1
236673,1
125813,1


In [84]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509791 entries, 0 to 509790
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   index                509791 non-null  int64  
 1   game_id              509791 non-null  int64  
 2   game_name            509790 non-null  object 
 3   first_release_date   365605 non-null  float64
 4   created_at_x         509791 non-null  int64  
 5   critic_rating        67467 non-null   float64
 6   critic_rating_count  67467 non-null   float64
 7   igdb_rating          104631 non-null  float64
 8   igdb_rating_count    104631 non-null  float64
 9   game_engine_id       70855 non-null   object 
 10  game_mode_id         333694 non-null  object 
 11  genre_id             435841 non-null  object 
 12  platform_id          384565 non-null  object 
 13  updated_at_x         509791 non-null  int64  
 14  external_game_id     505199 non-null  object 
 15  category_x       

In [64]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296516 entries, 0 to 296515
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   game_id              296516 non-null  int64  
 1   game_name            296515 non-null  object 
 2   first_release_date   202896 non-null  float64
 3   created_at           296516 non-null  int64  
 4   critic_rating        15912 non-null   float64
 5   critic_rating_count  15912 non-null   float64
 6   igdb_rating          31184 non-null   float64
 7   igdb_rating_count    31184 non-null   float64
 8   game_engine_id       31054 non-null   object 
 9   game_mode_id         174461 non-null  object 
 10  genre_id             244385 non-null  object 
 11  platform_id          216936 non-null  object 
 12  updated_at           296516 non-null  int64  
 13  external_game_id     274020 non-null  object 
 14  category             296516 non-null  int64  
dtypes: float64(5), in

In [77]:
external_games_df['game'].value_counts()

,count
game,
154986,93
205780,78
242493,76
256092,68
207393,67
...,...
168487,1
168484,1
168482,1


In [78]:
merged['game_id'].value_counts()

,count
game_id,
154986,93
205780,78
242493,76
256092,68
207393,67
...,...
213852,1
183381,1
209157,1


In [53]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296516 entries, 0 to 296515
Data columns (total 56 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       296516 non-null  int64  
 1   age_ratings              65865 non-null   object 
 2   alternative_names        66500 non-null   object 
 3   category                 296516 non-null  int64  
 4   cover                    231491 non-null  float64
 5   created_at               296516 non-null  int64  
 6   external_games           274020 non-null  object 
 7   first_release_date       202896 non-null  float64
 8   game_modes               174461 non-null  object 
 9   genres                   244385 non-null  object 
 10  involved_companies       135168 non-null  object 
 11  keywords                 100929 non-null  object 
 12  name                     296515 non-null  object 
 13  platforms                216936 non-null  object 
 14  play

In [52]:
games_df.columns

Index(['id', 'age_ratings', 'alternative_names', 'category', 'cover',
       'created_at', 'external_games', 'first_release_date', 'game_modes',
       'genres', 'involved_companies', 'keywords', 'name', 'platforms',
       'player_perspectives', 'release_dates', 'screenshots', 'similar_games',
       'slug', 'storyline', 'summary', 'tags', 'themes', 'updated_at', 'url',
       'videos', 'websites', 'checksum', 'game_localizations',
       'aggregated_rating', 'aggregated_rating_count', 'bundles', 'rating',
       'rating_count', 'total_rating', 'total_rating_count', 'ports',
       'language_supports', 'collections', 'artworks', 'hypes', 'status',
       'franchises', 'parent_game', 'game_engines', 'version_parent',
       'version_title', 'dlcs', 'multiplayer_modes', 'franchise', 'expansions',
       'remakes', 'expanded_games', 'remasters', 'forks',
       'standalone_expansions'],
      dtype='object')

In [75]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503336 entries, 0 to 503335
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   index                503336 non-null  int64  
 1   game_id              503336 non-null  int64  
 2   game_name            503335 non-null  object 
 3   first_release_date   361478 non-null  float64
 4   created_at_x         503336 non-null  int64  
 5   critic_rating        67378 non-null   float64
 6   critic_rating_count  67378 non-null   float64
 7   igdb_rating          104526 non-null  float64
 8   igdb_rating_count    104526 non-null  float64
 9   game_engine_id       70108 non-null   object 
 10  game_mode_id         329974 non-null  object 
 11  genre_id             430959 non-null  object 
 12  platform_id          379393 non-null  object 
 13  updated_at_x         503336 non-null  int64  
 14  external_game_id     503336 non-null  object 
 15  category_x       

In [76]:
not_in_external_games.head()

,id,age_ratings,alternative_names,category,cover,created_at,external_games,first_release_date,game_modes,genres,...,version_title,dlcs,multiplayer_modes,franchise,expansions,remakes,expanded_games,remasters,forks,standalone_expansions
2,88308,NaN,NaN,0,64849.0,1519237439,"[243459, 1960113]",1.478650e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,165668,NaN,NaN,0,255062.0,1630482010,"[2084097, 2115290]",NaN,NaN,[31],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,95080,NaN,NaN,0,NaN,1521818623,[1989881],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,89616,NaN,NaN,0,192106.0,1519986874,"[269608, 1949905]",NaN,[1],"[9, 33]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,91579,NaN,NaN,0,NaN,1521138594,"[128240, 1189096]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
not_in_games.head()

,id,category,created_at,game,name,uid,updated_at,url,year,checksum
1,2434584,1,1650877954,199317,Decorporation,1968950,1651487795,https://store.steampowered.com/app/1968950,2022.0,16e28cac-1b7d-7c4c-c283-d1e704e53b57
2,1601538,1,1537599485,110092,Devil Engine,891790,1633088635,https://store.steampowered.com/app/891790,2018.0,ce5b8084-fa84-5d6e-de93-ec4267ae79ec
3,1187074,14,1480340090,11697,Zettai Zetsumei Toshi 3: Kowareyuku Machi to K...,22819,1551883771,https://www.twitch.tv/directory/game/Zettai Ze...,NaN,864de74c-08b5-77a4-9664-d023ee08af3f
4,1601549,1,1537600739,110088,Generation Streets,887970,1633090129,https://store.steampowered.com/app/887970,2018.0,130a0688-5a02-cb15-6fe4-dd77266ae030
5,1710320,1,1551154010,115574,Symbitic Love,948830,1633091081,https://store.steampowered.com/app/948830,2019.0,c850074e-d0ea-1ffb-00b2-973965440947


In [38]:
games_df[games_df['id'] == 1601538]

,id,age_ratings,alternative_names,category,cover,created_at,external_games,first_release_date,game_modes,genres,...,version_title,dlcs,multiplayer_modes,franchise,expansions,remakes,expanded_games,remasters,forks,standalone_expansions


In [33]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752320 entries, 0 to 752319
Data columns (total 65 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       752320 non-null  int64  
 1   age_ratings              65865 non-null   object 
 2   alternative_names        66500 non-null   object 
 3   category_x               296516 non-null  float64
 4   cover                    231491 non-null  float64
 5   created_at_x             296516 non-null  float64
 6   external_games           274020 non-null  object 
 7   first_release_date       202896 non-null  float64
 8   game_modes               174461 non-null  object 
 9   genres                   244385 non-null  object 
 10  involved_companies       135168 non-null  object 
 11  keywords                 100929 non-null  object 
 12  name_x                   296515 non-null  object 
 13  platforms                216936 non-null  object 
 14  play

In [31]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296516 entries, 0 to 296515
Data columns (total 56 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       296516 non-null  int64  
 1   age_ratings              65865 non-null   object 
 2   alternative_names        66500 non-null   object 
 3   category                 296516 non-null  int64  
 4   cover                    231491 non-null  float64
 5   created_at               296516 non-null  int64  
 6   external_games           274020 non-null  object 
 7   first_release_date       202896 non-null  float64
 8   game_modes               174461 non-null  object 
 9   genres                   244385 non-null  object 
 10  involved_companies       135168 non-null  object 
 11  keywords                 100929 non-null  object 
 12  name                     296515 non-null  object 
 13  platforms                216936 non-null  object 
 14  play

In [21]:
external_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548393 entries, 0 to 548392
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          548393 non-null  int64  
 1   category    548393 non-null  int64  
 2   created_at  548393 non-null  int64  
 3   game        548393 non-null  int64  
 4   name        536287 non-null  object 
 5   uid         548393 non-null  object 
 6   updated_at  548393 non-null  int64  
 7   url         518902 non-null  object 
 8   year        169423 non-null  float64
 9   checksum    548393 non-null  object 
dtypes: float64(1), int64(5), object(4)
memory usage: 41.8+ MB


In [20]:
external_games_df[external_games_df['uid'] == 'B09BDPXS9H']

,id,category,created_at,game,name,uid,updated_at,url,year,checksum
288011,2170826,20,1638784275,153640,Shin Megami Tensei V Fall of Man Premium Editi...,B09BDPXS9H,1638789078,https://amazon.co.uk/dp/B09BDPXS9H,NaN,4e3a5534-e9ce-c103-ab84-fdf145af29ec
356663,2452640,20,1656335044,153640,Shin Megami Tensei V Collectors Edition,B09BDPXS9H,1656345771,https://amazon.es/dp/B09BDPXS9H,NaN,91960010-43f5-9d3c-d16e-2a95647e6d6d
356664,2452639,20,1656335044,153640,Shin Megami Tensei V Edition Premium Chute de ...,B09BDPXS9H,1656345718,https://amazon.fr/dp/B09BDPXS9H,NaN,d46cef40-49f9-8b13-0098-7def6f378a8e
449176,2703675,20,1684199794,153640,Shin Megami Tensei V (Premium Edition),B09BDPXS9H,1684204930,https://amazon.it/dp/B09BDPXS9H,NaN,9b8d51a4-6063-e059-0b14-4fd8412c7d93
449220,2703674,20,1684199794,153640,Shin Megami Tensei V Fall of Man Premium Editi...,B09BDPXS9H,1684206194,https://amazon.co.uk/dp/B09BDPXS9H,NaN,06282c70-5074-20fc-25d5-f46ad5dd3442
449517,2703678,20,1684199794,153640,Shin Megami Tensei V Collectors Edition,B09BDPXS9H,1684293150,https://amazon.es/dp/B09BDPXS9H,NaN,68c9efef-6c94-20af-4445-40c63c07b960
449523,2703677,20,1684199794,153640,Shin Megami Tensei V Edition Premium Chute de ...,B09BDPXS9H,1684293665,https://amazon.fr/dp/B09BDPXS9H,NaN,fc91ea29-edf7-e69c-14c8-83f64b61a8ad
449614,2703676,20,1684199794,153640,Shin Megami Tensei V Premium Edition [Nintendo...,B09BDPXS9H,1684293932,https://amazon.de/dp/B09BDPXS9H,NaN,fa9a9ab9-5cc0-5f4c-b162-be5e9a9ae3b4
490695,2911101,20,1714384333,153640,Shin Megami Tensei V (Premium Edition),B09BDPXS9H,1715222364,https://amazon.it/dp/B09BDPXS9H,NaN,921d9fb6-a234-8059-53af-836a90d8cccf
491418,2911104,20,1714384334,153640,Shin Megami Tensei V Edition Premium Chute de ...,B09BDPXS9H,1715224961,https://amazon.fr/dp/B09BDPXS9H,NaN,22522e27-c877-7972-8a50-cba4c278e722


In [29]:
external_games_df[external_games_df['id'].isin([1141206,1951206])]

,id,category,created_at,game,name,uid,updated_at,url,year,checksum
90509,1141206,13,1530144000,104294,Rummy +,790729888,1530144000,https://itunes.apple.com/us/app/rummy/id790729...,2014.0,502860b8-cdb9-3852-a30c-873a5b6e51a9
224804,1951206,14,1604620800,112861,Dark Roll,747613408,1604620800,https://www.twitch.tv/directory/game/Dark%20Roll,NaN,1be550be-b5bd-4d1e-496d-28f16fd9d55c
548391,1141206,13,1530170731,104294,Rummy +,790729888,1530176006,https://itunes.apple.com/us/app/rummy/id790729...,2014.0,42ca41ec-6f8d-7432-7e1d-b06c9ff1830f
548392,1951206,14,1604663380,112861,Dark Roll,747613408,1604663380,https://www.twitch.tv/directory/game/Dark%20Roll,NaN,ffbb2162-5614-53e7-fba7-955467e4cb2b


In [27]:
external_games_df['id'].value_counts().reset_index()

,id,count
0,1141206,2
1,1951206,2
2,132341,1
3,2524708,1
4,2524940,1
...,...,...
548386,2515583,1
548387,1920114,1
548388,1920149,1
548389,1920393,1


In [27]:
external_games_df[external_games_df['category'] == 1].info()

<class 'pandas.core.frame.DataFrame'>
Index: 126975 entries, 1 to 548317
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          126975 non-null  int64  
 1   category    126975 non-null  int64  
 2   created_at  126975 non-null  int64  
 3   game        126975 non-null  int64  
 4   name        126975 non-null  object 
 5   uid         126975 non-null  object 
 6   updated_at  126975 non-null  int64  
 7   url         107302 non-null  object 
 8   year        75904 non-null   float64
 9   checksum    126975 non-null  object 
dtypes: float64(1), int64(5), object(4)
memory usage: 10.7+ MB


In [28]:
steam_games = external_games_df[external_games_df['category'] == 1]

In [ ]:
steam_games['uid'].astype(int)

In [36]:
steam_games_app_ids = list(steam_games['uid'].drop_duplicates().astype(int))

In [8]:
api_key = 'ccb1befb418f37929875cf0f25886e7d5b3fc673'

In [9]:
end_point = 'https://api.isthereanydeal.com/games/lookup/v1'
header = {
    'Client-ID':'3d88414c978627a1',
    'Authorization':api_key
}

In [3]:
all_games = []

In [ ]:
# Search for Valheim by title


In [63]:
# Pass in a steam app ID to retrieve game ID and title
for id in steam_games_app_ids:

  try:
    # appid: the steam app ID
    # api_key: the API key from IsThereAnyDeal
    params = {'appid':id, 'key':api_key}

    # Make the API call and validate response status
    r = requests.get(end_point, headers = header, params = params)
    if r.status_code != 200:
      raise Exception(f"API call failed with status code {r.status_code}: {r.text}")

    # Print confirmation
    print(f"Getting the results for {r.url}")

    # Parse JSON response
    results = r.json()
    # Get only the game ID
    game = results['game']['id']
    # Add game to master list
    all_games.append(game)

  # Stop the loop on network failure
  except requests.exceptions.RequestException as e:
    print(f"Network-related error occurred: {e}")
    break

  # Stop the loop on unexpected errors
  except Exception as e:
    print(f"An error occurred: {e}")
    break

Getting the results for https://api.isthereanydeal.com/games/lookup/v1?appid=1968950&key=ccb1befb418f37929875cf0f25886e7d5b3fc673
Getting the results for https://api.isthereanydeal.com/games/lookup/v1?appid=891790&key=ccb1befb418f37929875cf0f25886e7d5b3fc673
Getting the results for https://api.isthereanydeal.com/games/lookup/v1?appid=887970&key=ccb1befb418f37929875cf0f25886e7d5b3fc673
Getting the results for https://api.isthereanydeal.com/games/lookup/v1?appid=948830&key=ccb1befb418f37929875cf0f25886e7d5b3fc673
Getting the results for https://api.isthereanydeal.com/games/lookup/v1?appid=940070&key=ccb1befb418f37929875cf0f25886e7d5b3fc673
Getting the results for https://api.isthereanydeal.com/games/lookup/v1?appid=846490&key=ccb1befb418f37929875cf0f25886e7d5b3fc673
Getting the results for https://api.isthereanydeal.com/games/lookup/v1?appid=895670&key=ccb1befb418f37929875cf0f25886e7d5b3fc673
Getting the results for https://api.isthereanydeal.com/games/lookup/v1?appid=692360&key=ccb1befb

KeyboardInterrupt: 

0

In [39]:
for id in steam_games_app_ids[0:5]:


1968950
891790
887970
948830
940070
